In [22]:
import pandas as pd
import numpy as np
import math

Read in data and parent child matrices

In [23]:
clean_ppi_q_by_digit_sheets = ['AGG', 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0]
parent_child_matrices_sheets = [4, 5, 6, 7, 8, 9, 10, 11, 12]

In [24]:
clean_ppi_q_by_digit = {}

for i in clean_ppi_q_by_digit_sheets:
    clean_ppi_q_by_digit[i] = pd.read_excel('clean_ppi_q_by_digit.xlsx', sheet_name = str(i), index_col=0)

parent_child_matrices = {}

for i in parent_child_matrices_sheets:
    parent_child_matrices[i] = pd.read_excel('parent_child_matrices.xlsx', sheet_name = str(i), index_col=0)

In [25]:
for i in clean_ppi_q_by_digit.keys():
    clean_ppi_q_by_digit[i] = clean_ppi_q_by_digit[i].drop('YYYYQQ',axis=1)

In [26]:
for i in parent_child_matrices.keys():
    parent_child_matrices[i] = parent_child_matrices[i].drop('sum',axis=1)

In [51]:
covariates = pd.read_csv('all_covariates.csv', index_col=0)

In [53]:
# convert index to periods
covariates.index = pd.period_range(start='1980-01-01', end = '2050-12-31', freq='Q')
# reindexto 1960
covariates = covariates.reindex(pd.period_range(start='1960-01-01', end = '2050-12-31', freq='Q'))

In [102]:
# remove trailing spaces
covariates.columns = covariates.columns.str.strip()

concatenate all digit level dataframes into one master df

In [27]:
all_df = pd.concat(clean_ppi_q_by_digit,axis=1)
all_df.columns = all_df.columns.droplevel(0)

In [28]:
all_df.index = all_df.index.to_period()

Create panel dataset

In [29]:
panel = all_df.melt()

In [30]:
# repeat the index as many times as number of series
panel.index = list(all_df.index) * len(all_df.columns)

In [31]:
panel['digit'] = np.nan

In [32]:
# digit to code dictionary
digit_dict = {}
for i in clean_ppi_q_by_digit.keys():
    digit_dict[i] = list(clean_ppi_q_by_digit[i].columns)

In [33]:
digit_dict[5]

['PCU11331-11331-',
 'PCU21211-21211-',
 'PCU21221-21221-',
 'PCU21223-21223-',
 'PCU21229-21229-',
 'PCU21231-21231-',
 'PCU21232-21232-',
 'PCU21239-21239-',
 'PCU21311-21311-',
 'PCU22111-22111-',
 'PCU22112-22112-',
 'PCU22121-22121-',
 'PCU31111-31111-',
 'PCU31121-31121-',
 'PCU31122-31122-',
 'PCU31123-31123-',
 'PCU31131-31131-',
 'PCU31134-31134-',
 'PCU31135-31135-',
 'PCU31141-31141-',
 'PCU31142-31142-',
 'PCU31151-31151-',
 'PCU31152-31152-',
 'PCU31161-31161-',
 'PCU31171-31171-',
 'PCU31181-31181-',
 'PCU31182-31182-',
 'PCU31183-31183-',
 'PCU31191-31191-',
 'PCU31192-31192-',
 'PCU31193-31193-',
 'PCU31194-31194-',
 'PCU31199-31199-',
 'PCU31211-31211-',
 'PCU31212-31212-',
 'PCU31213-31213-',
 'PCU31214-31214-',
 'PCU31223-31223-',
 'PCU31311-31311-',
 'PCU31321-31321-',
 'PCU31322-31322-',
 'PCU31323-31323-',
 'PCU31324-31324-',
 'PCU31331-31331-',
 'PCU31332-31332-',
 'PCU31411-31411-',
 'PCU31412-31412-',
 'PCU31491-31491-',
 'PCU31499-31499-',
 'PCU31521-31521-',


In [34]:
# reverse dictionary to get code to digit mapping
code_digit_dict = {}
for i,j in digit_dict.items():
    for k in j:
        code_digit_dict[k] = i

In [35]:
for i in digit_dict.keys():
    for j in digit_dict[i]:
        panel.loc[panel['variable'] == j, 'digit'] = i

In [36]:
# create outcome column
panel['outcome'] = np.nan
panel.loc[panel['digit']==6, 'outcome'] = 1
panel.loc[panel['digit']!=6, 'outcome'] = 0

In [37]:
panel['parent1'] = np.nan

In [38]:
for i in panel['variable'].unique():
    # if code is an aggregate or 3 digit, there is no aggregate available
    if code_digit_dict[i] == 'AGG' or code_digit_dict[i] == 3:
        continue
    else:
        # if code has no direct parent leave as NA - it is the top level code
        if parent_child_matrices[code_digit_dict[i]].T[parent_child_matrices[code_digit_dict[i]].T.loc[:,i] == 1][i].index.values.size==0:
            continue
        # find parent through the parent_child_matrices
        else:
            panel.loc[panel['variable'] == i, 'parent1'] = parent_child_matrices[code_digit_dict[i]].T[parent_child_matrices[code_digit_dict[i]].T.loc[:,i] == 1][i].index.values[0]

In [39]:
# counter for parent level
c = 2

# dictionary to store removed parents during data clean up
parents_removed = {}

# find list of children in the previous level
children = list(panel['parent'+str(c-1)].unique())

# remove nan values from the list
for i in children:
    if type(i) != str:
        if math.isnan(i):
            children.remove(i)

# while there are possible parents
while len(children) > 0:
    print(c)
    parents_removed[c] = []
    # set new column to nan
    panel['parent'+str(c)] = np.nan

    for i in children:
        # some parent codes will have been removed in the data cleanup - this try/except records these
        try:
            # if code is an aggregate or 3 digit, there is no aggregate available
            if code_digit_dict[i] == 'AGG' or code_digit_dict[i] == 3:
                continue
            else:
                # if code has no direct parent leave as NA - it is the top level code
                if parent_child_matrices[code_digit_dict[i]].T[parent_child_matrices[code_digit_dict[i]].T.loc[:,i] == 1][i].index.values.size==0:
                    continue
                # find parent through the parent_child_matrices
                else:
                    panel.loc[panel['parent'+str(c-1)]==i, 'parent'+str(c)] = parent_child_matrices[code_digit_dict[i]].T[parent_child_matrices[code_digit_dict[i]].T.loc[:,i]==1][i].index.values[0]
        except KeyError:
            parents_removed[c].append(i)

    # increase counter and recreate children list
    c += 1

    # find list of children in the previous level
    children = list(panel['parent'+str(c-1)].unique())

    # remove nan values from the list
    for i in children:
        if type(i) != str:
            if math.isnan(i):
                children.remove(i)

2
3
4
5
6
7


In [40]:
parents_removed

{2: ['PCU42450-42450-',
  'PCU33299T33299T4',
  'PCU3335193335198',
  'PCU42993042993022'],
 3: [],
 4: [],
 5: [],
 6: [],
 7: []}

# Add covariates

In [113]:
cov_map = pd.read_csv('6digit_covariates.csv', index_col=0)

In [114]:
cov_map

,PMx,COSTx,PENGx,AWAGESx,XEMPEx,GY$x
PCU113310113310,PMA,COSTTOT,PENGA,-,XEMPE1133,GY$11ZZZZZZZZ
PCU211111211111,PMOIL,COSTTOT,PENGE,AWAGES21,XEMPE211,GY$211ZZZZZZZ
PCU211112211112,PMOIL,COSTTOT,PENGE,AWAGES21,XEMPE211,GY$211112ZZZZ
PCU212210212210,PMOTHE,COSTTOT,PENGE,AWAGES21,XEMPE2122,GY$2122ZZZZZZ
PCU212230212230,PMOTHE,COSTTOT,PENGE,AWAGES21,XEMPE2122,GY$2122ZZZZZZ
...,...,...,...,...,...,...
PCU713940713940,PMOTHS,COSTTOT,PENGPUBLIC,AWAGES71,XEMPE7139,GY$7139ZZZZZZ
PCU721110721110,PMHOTL,COSTTOT,PENGCOMMER,AWAGES72,XEMPE721,GY$721ZZZZZZZ
PCU721120721120,PMHOTL,COSTTOT,PENGCOMMER,AWAGES72,XEMPE721,GY$721ZZZZZZZ
PCU811310811310,PMREPAIR,COSTOTHFMRE,PENGMOTH,AWAGES81,XEMPE8113,GY$8113ZZZZZZ


In [115]:
for i in cov_map.columns:
    panel[i] = np.nan
    for j in cov_map.index:
        if cov_map.loc[j,i] == "-":
            continue
        else:
            panel.loc[panel['variable'] == j, i] = covariates.loc[:,cov_map.loc[j,i]]

In [116]:
panel.to_csv('panel.csv')